In [ ]:
import os
import torch
import gym
import numpy as np
from TD3_torch.TD3 import TD3
from PIL import Image
from TD3_torch.utils import ReplayBuffer

env_name = 'BipedalWalkerHardcore-v2'
learning_rate_base = 0.0001
log_interval = 10           # print avg reward after interval
random_seed = 333
gamma = 0.99                # discount for future rewards
batch_size = 1024        # num of transitions sampled from replay buffer
exploration_noise_base = 0.3 
polyak_int = [0.9999, 0.999999]              # target policy update parameter (1-tau)
policy_noise = 0.2          # target policy smoothing noise
noise_clip = 0.5
policy_delay = 2            # delayed policy updates parameter
max_episodes = 100000         # max num of episodes
max_timesteps = 3000        # max timesteps in one episode
max_buffer_length = 2000000
directory = "./preTrained/td3_torch/{}".format(env_name) # save trained models
filename = "TD3_torch_{}_{}".format(env_name, random_seed)
reward_history = []



def train():
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    polyak = polyak_int[0]
    exploration_noise = exploration_noise_base
    
    actor_config = [
        {'dim': (state_dim, 256), 'dropout': False, 'activation': 'relu'},
        {'dim': (256, 256), 'dropout': True, 'activation':'relu'},
        {'dim': (256, 128), 'dropout': True, 'activation': 'relu'},
        {'dim': (128, 64), 'dropout': False, 'activation': 'relu'},
        {'dim': (64, action_dim),'dropout': False, 'activation': False}
    ]
    
    critic_config = [
        {'dim': (state_dim + action_dim, 256), 'dropout': False, 'activation': 'relu'},
        {'dim': (256, 256), 'dropout': False , 'activation':'relu'},
        {'dim': (256, 128), 'dropout': False, 'activation': 'relu'},
        {'dim': (128, 64), 'dropout': False, 'activation': 'relu'},
        {'dim': (64, 1), 'dropout': False, 'activation': False},
    ]
    
    policy = TD3(actor_config, critic_config, max_action, lr=learning_rate_base)   
    replay_buffer = ReplayBuffer(max_length=max_buffer_length)
    
    print("action_space={}".format(env.action_space))
    print("obs_space={}".format(env.observation_space))
    print("threshold={} \n".format(env.spec.reward_threshold))     
    
    if random_seed:
        print("Random Seed: {}".format(random_seed))
        env.seed(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
    
    # loading models
    policy.load(directory, filename)
    
    # logging variables:        
    log_f = open("log.txt","w+")
    
    # training procedure:
    for episode in range(1, max_episodes+1):
        ep_reward = 0
        state = env.reset()
       
        for t in range(max_timesteps):
            # select action and add exploration noise:
            action = policy.select_action(state)
            action = action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0])
            action = action.clip(env.action_space.low, env.action_space.high)
            
            # take action in env:
            next_state, reward, done, _ = env.step(action)
            replay_buffer.add((state, action, reward, next_state, float(done)))
            state = next_state
            
            ep_reward += reward
            
            # if episode is done then update policy:
            if done or t==(max_timesteps-1):
                policy.update(replay_buffer, t, batch_size, gamma, polyak, policy_noise, noise_clip, policy_delay)
                break
        
        reward_history.append(ep_reward)
        avg_reward = np.mean(reward_history[-100:]) 
        
        # logging updates:        
        log_f.write('{},{}\n'.format(episode, ep_reward))
        log_f.flush()
       
        
        # if avg reward > 300 then save and stop traning:
        if avg_reward >= env.spec.reward_threshold: 
            print("########## Solved! ###########")
            name = filename + '_solved'
            policy.save(directory, name)
            log_f.close()
            break
            
        # Calculate polyak
        #part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
        #if part > 1:
        #    part = 1
        #polyak = polyak_int[0] + (1 - part) * (polyak_int[1] - polyak_int[0])     
        
        # Calculate LR
        part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
        if part > 1:
            part = 1
        learning_rate = learning_rate_base - learning_rate_base * (1 - part) * 0.9
        policy.set_optimizers(lr=learning_rate)
        
        # Calculate Exploration Noise
        exploration_noise = exploration_noise_base - exploration_noise_base * (1 - part) * 0.9
        
        
        if episode > 500:
            policy.save(directory, filename)
        
        # print avg reward every log interval:
        if episode % log_interval == 0:            
            print("Ep:{}   Rew:{:3.2f}  Avg Rew:{:3.2f}  LR:{:8.8f}   Polyak:{:5.5f}  Bf:{:2.0f}  EN:{:0.4f}  Loss: {:5.3f} {:5.3f} {:5.3f}".format(
                episode, ep_reward, avg_reward, learning_rate, polyak, replay_buffer.get_fill(), exploration_noise, policy.actor_loss, policy.loss_Q1, policy.loss_Q2))

train()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
ACTOR=Sequential(
  (0): Linear(in_features=24, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Dropout(p=0.2)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=128, bias=True)
  (6): Dropout(p=0.2)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=64, bias=True)
  (9): ReLU()
  (10): Linear(in_features=64, out_features=4, bias=True)
)
ACTOR=Sequential(
  (0): Linear(in_features=24, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Dropout(p=0.2)
  (4): ReLU()
  (5): Linear(in_features=256, out_features=128, bias=True)
  (6): Dropout(p=0.2)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=64, bias=True)
  (9): ReLU()
  (10): Linear(in_features=64,

Ep:490   Rew:-109.78  Avg Rew:-111.50  LR:0.00009230   Polyak:0.99990  Bf: 4  EN:0.2769  Loss: 0.632 21.875 13.009
Ep:500   Rew:-113.68  Avg Rew:-111.51  LR:0.00009230   Polyak:0.99990  Bf: 4  EN:0.2769  Loss: -0.020 4.254 4.161
Ep:510   Rew:-109.18  Avg Rew:-111.43  LR:0.00009229   Polyak:0.99990  Bf: 4  EN:0.2769  Loss: 0.602 6.577 9.599
Ep:520   Rew:-111.38  Avg Rew:-111.15  LR:0.00009223   Polyak:0.99990  Bf: 4  EN:0.2767  Loss: 0.135 7.114 5.196
Ep:530   Rew:-112.15  Avg Rew:-111.20  LR:0.00009224   Polyak:0.99990  Bf: 4  EN:0.2767  Loss: 0.378 3.695 3.987
Ep:540   Rew:-115.18  Avg Rew:-111.33  LR:0.00009227   Polyak:0.99990  Bf: 4  EN:0.2768  Loss: 0.112 6.328 3.949
Ep:550   Rew:-112.11  Avg Rew:-111.48  LR:0.00009230   Polyak:0.99990  Bf: 4  EN:0.2769  Loss: 0.328 9.820 2.057
Ep:560   Rew:-109.58  Avg Rew:-111.40  LR:0.00009228   Polyak:0.99990  Bf: 4  EN:0.2768  Loss: 0.637 4.085 4.227
Ep:570   Rew:-108.99  Avg Rew:-111.46  LR:0.00009229   Polyak:0.99990  Bf: 4  EN:0.2769  Loss

Ep:1220   Rew:-120.67  Avg Rew:-108.95  LR:0.00009179   Polyak:0.99990  Bf: 8  EN:0.2754  Loss: 4.827 1.990 1.993
Ep:1230   Rew:-108.91  Avg Rew:-109.96  LR:0.00009199   Polyak:0.99990  Bf: 8  EN:0.2760  Loss: 4.673 2.880 3.343
Ep:1240   Rew:-117.60  Avg Rew:-110.87  LR:0.00009217   Polyak:0.99990  Bf: 8  EN:0.2765  Loss: 4.792 1.323 1.321
Ep:1250   Rew:-121.92  Avg Rew:-112.20  LR:0.00009244   Polyak:0.99990  Bf: 8  EN:0.2773  Loss: 4.846 2.910 2.283
Ep:1260   Rew:-120.78  Avg Rew:-113.02  LR:0.00009260   Polyak:0.99990  Bf: 8  EN:0.2778  Loss: 5.954 2.976 4.004
Ep:1270   Rew:-106.79  Avg Rew:-113.74  LR:0.00009275   Polyak:0.99990  Bf: 8  EN:0.2782  Loss: 4.912 3.180 2.404
Ep:1280   Rew:-113.19  Avg Rew:-113.87  LR:0.00009277   Polyak:0.99990  Bf: 8  EN:0.2783  Loss: 5.547 2.113 1.955
Ep:1290   Rew:-141.33  Avg Rew:-115.10  LR:0.00009302   Polyak:0.99990  Bf: 8  EN:0.2791  Loss: 4.435 3.003 2.149
Ep:1300   Rew:-116.16  Avg Rew:-115.60  LR:0.00009312   Polyak:0.99990  Bf: 8  EN:0.2794

Ep:1940   Rew:-127.52  Avg Rew:-117.56  LR:0.00009351   Polyak:0.99990  Bf:15  EN:0.2805  Loss: 8.119 1.627 1.561
Ep:1950   Rew:-130.56  Avg Rew:-118.49  LR:0.00009370   Polyak:0.99990  Bf:15  EN:0.2811  Loss: 7.797 2.135 2.370
Ep:1960   Rew:-120.75  Avg Rew:-118.68  LR:0.00009374   Polyak:0.99990  Bf:15  EN:0.2812  Loss: 7.559 2.269 1.969
Ep:1970   Rew:-115.44  Avg Rew:-119.14  LR:0.00009383   Polyak:0.99990  Bf:15  EN:0.2815  Loss: 8.908 2.381 2.087
Ep:1980   Rew:-101.90  Avg Rew:-119.06  LR:0.00009381   Polyak:0.99990  Bf:15  EN:0.2814  Loss: 9.112 2.172 1.524
Ep:1990   Rew:-105.91  Avg Rew:-118.24  LR:0.00009365   Polyak:0.99990  Bf:15  EN:0.2809  Loss: 7.800 2.253 2.236
Ep:2000   Rew:-127.32  Avg Rew:-118.50  LR:0.00009370   Polyak:0.99990  Bf:15  EN:0.2811  Loss: 8.007 1.883 1.260
Ep:2010   Rew:-104.99  Avg Rew:-117.80  LR:0.00009356   Polyak:0.99990  Bf:15  EN:0.2807  Loss: 8.402 2.922 3.160
Ep:2020   Rew:-117.41  Avg Rew:-117.02  LR:0.00009340   Polyak:0.99990  Bf:15  EN:0.2802

Ep:2660   Rew:-96.22  Avg Rew:-107.56  LR:0.00009151   Polyak:0.99990  Bf:25  EN:0.2745  Loss: 9.303 1.626 1.656
Ep:2670   Rew:-98.57  Avg Rew:-108.90  LR:0.00009178   Polyak:0.99990  Bf:25  EN:0.2753  Loss: 11.643 2.431 2.528
Ep:2680   Rew:-125.03  Avg Rew:-109.43  LR:0.00009189   Polyak:0.99990  Bf:25  EN:0.2757  Loss: 11.317 1.851 1.883
Ep:2690   Rew:-107.62  Avg Rew:-108.74  LR:0.00009175   Polyak:0.99990  Bf:26  EN:0.2752  Loss: 10.488 2.088 1.602
Ep:2700   Rew:-100.02  Avg Rew:-107.95  LR:0.00009159   Polyak:0.99990  Bf:26  EN:0.2748  Loss: 11.668 2.034 1.677
Ep:2710   Rew:-124.88  Avg Rew:-107.26  LR:0.00009145   Polyak:0.99990  Bf:26  EN:0.2744  Loss: 12.590 2.212 1.705
Ep:2720   Rew:-93.88  Avg Rew:-107.69  LR:0.00009154   Polyak:0.99990  Bf:26  EN:0.2746  Loss: 11.800 1.535 1.415
Ep:2730   Rew:-96.39  Avg Rew:-107.21  LR:0.00009144   Polyak:0.99990  Bf:27  EN:0.2743  Loss: 11.075 3.775 3.058
Ep:2740   Rew:-81.22  Avg Rew:-106.56  LR:0.00009131   Polyak:0.99990  Bf:27  EN:0.27

...

Ep:5550   Rew:-13.22  Avg Rew:-62.25  LR:0.00008245   Polyak:0.99990  Bf:100  EN:0.2474  Loss: 6.722 0.259 0.369
Ep:5560   Rew:0.66  Avg Rew:-60.23  LR:0.00008205   Polyak:0.99990  Bf:100  EN:0.2461  Loss: 6.338 0.149 0.170
Ep:5570   Rew:12.54  Avg Rew:-60.52  LR:0.00008210   Polyak:0.99990  Bf:100  EN:0.2463  Loss: 6.589 0.416 0.542
Ep:5580   Rew:-79.65  Avg Rew:-60.41  LR:0.00008208   Polyak:0.99990  Bf:100  EN:0.2462  Loss: 6.491 0.088 0.105
Ep:5590   Rew:-21.21  Avg Rew:-60.68  LR:0.00008214   Polyak:0.99990  Bf:100  EN:0.2464  Loss: 6.044 0.148 0.115
Ep:5600   Rew:-40.00  Avg Rew:-60.83  LR:0.00008217   Polyak:0.99990  Bf:100  EN:0.2465  Loss: 6.584 0.482 0.494
Ep:5610   Rew:-93.51  Avg Rew:-62.41  LR:0.00008248   Polyak:0.99990  Bf:100  EN:0.2474  Loss: 6.582 0.248 0.227
Ep:5620   Rew:-55.82  Avg Rew:-58.87  LR:0.00008177   Polyak:0.99990  Bf:100  EN:0.2453  Loss: 6.648 0.207 0.173
Ep:5630   Rew:-91.20  Avg Rew:-58.83  LR:0.00008177   Polyak:0.99990  Bf:100  EN:0.2453  Loss: 6.20

Ep:6280   Rew:-65.42  Avg Rew:-56.94  LR:0.00008139   Polyak:0.99990  Bf:100  EN:0.2442  Loss: 6.039 0.156 0.205
Ep:6290   Rew:-41.89  Avg Rew:-56.65  LR:0.00008133   Polyak:0.99990  Bf:100  EN:0.2440  Loss: 6.689 0.213 0.276
Ep:6300   Rew:-66.51  Avg Rew:-57.25  LR:0.00008145   Polyak:0.99990  Bf:100  EN:0.2444  Loss: 6.789 0.451 0.546
Ep:6310   Rew:-131.91  Avg Rew:-58.08  LR:0.00008162   Polyak:0.99990  Bf:100  EN:0.2448  Loss: 6.442 0.224 0.312
Ep:6320   Rew:-60.39  Avg Rew:-59.04  LR:0.00008181   Polyak:0.99990  Bf:100  EN:0.2454  Loss: 6.667 0.380 0.643
Ep:6330   Rew:-100.91  Avg Rew:-58.72  LR:0.00008174   Polyak:0.99990  Bf:100  EN:0.2452  Loss: 6.503 0.481 0.461
Ep:6340   Rew:-67.37  Avg Rew:-59.68  LR:0.00008194   Polyak:0.99990  Bf:100  EN:0.2458  Loss: 5.982 0.477 0.452
Ep:6350   Rew:-61.62  Avg Rew:-59.54  LR:0.00008191   Polyak:0.99990  Bf:100  EN:0.2457  Loss: 6.618 0.240 0.310
Ep:6360   Rew:-71.23  Avg Rew:-61.25  LR:0.00008225   Polyak:0.99990  Bf:100  EN:0.2468  Loss:

Ep:7010   Rew:-88.16  Avg Rew:-47.33  LR:0.00007947   Polyak:0.99990  Bf:100  EN:0.2384  Loss: 5.438 0.223 0.254
Ep:7020   Rew:-66.40  Avg Rew:-46.06  LR:0.00007921   Polyak:0.99990  Bf:100  EN:0.2376  Loss: 5.989 0.187 0.218
Ep:7030   Rew:-124.29  Avg Rew:-46.03  LR:0.00007921   Polyak:0.99990  Bf:100  EN:0.2376  Loss: 5.804 0.440 0.368
Ep:7040   Rew:-75.60  Avg Rew:-45.38  LR:0.00007908   Polyak:0.99990  Bf:100  EN:0.2372  Loss: 5.718 0.529 0.323
Ep:7050   Rew:-72.82  Avg Rew:-45.82  LR:0.00007916   Polyak:0.99990  Bf:100  EN:0.2375  Loss: 5.634 0.101 0.147
Ep:7060   Rew:-50.44  Avg Rew:-50.69  LR:0.00008014   Polyak:0.99990  Bf:100  EN:0.2404  Loss: 5.582 0.398 0.428
Ep:7070   Rew:17.38  Avg Rew:-51.72  LR:0.00008034   Polyak:0.99990  Bf:100  EN:0.2410  Loss: 5.773 0.375 0.350
Ep:7080   Rew:-30.96  Avg Rew:-50.68  LR:0.00008014   Polyak:0.99990  Bf:100  EN:0.2404  Loss: 5.747 0.282 0.247
Ep:7090   Rew:-13.45  Avg Rew:-52.23  LR:0.00008045   Polyak:0.99990  Bf:100  EN:0.2413  Loss: 5

Ep:7740   Rew:-61.80  Avg Rew:-31.10  LR:0.00007622   Polyak:0.99990  Bf:100  EN:0.2287  Loss: 5.333 0.216 0.157
Ep:7750   Rew:-61.50  Avg Rew:-34.26  LR:0.00007685   Polyak:0.99990  Bf:100  EN:0.2306  Loss: 5.347 0.200 0.284
Ep:7760   Rew:-6.72  Avg Rew:-35.96  LR:0.00007719   Polyak:0.99990  Bf:100  EN:0.2316  Loss: 5.471 0.300 0.654
Ep:7770   Rew:-47.81  Avg Rew:-34.28  LR:0.00007686   Polyak:0.99990  Bf:100  EN:0.2306  Loss: 5.508 0.261 0.179
Ep:7780   Rew:-59.75  Avg Rew:-34.83  LR:0.00007697   Polyak:0.99990  Bf:100  EN:0.2309  Loss: 5.534 0.261 0.353
Ep:7790   Rew:30.05  Avg Rew:-32.12  LR:0.00007642   Polyak:0.99990  Bf:100  EN:0.2293  Loss: 5.533 0.283 0.375
Ep:7800   Rew:28.12  Avg Rew:-32.40  LR:0.00007648   Polyak:0.99990  Bf:100  EN:0.2294  Loss: 5.246 0.219 0.206
Ep:7810   Rew:-25.96  Avg Rew:-33.79  LR:0.00007676   Polyak:0.99990  Bf:100  EN:0.2303  Loss: 5.282 0.125 0.129
Ep:7820   Rew:10.71  Avg Rew:-29.08  LR:0.00007582   Polyak:0.99990  Bf:100  EN:0.2274  Loss: 5.410

Ep:8470   Rew:-103.33  Avg Rew:-11.40  LR:0.00007228   Polyak:0.99990  Bf:100  EN:0.2168  Loss: 4.966 0.170 0.285
Ep:8480   Rew:-16.21  Avg Rew:-10.55  LR:0.00007211   Polyak:0.99990  Bf:100  EN:0.2163  Loss: 4.909 0.267 0.242
Ep:8490   Rew:35.47  Avg Rew:-11.64  LR:0.00007233   Polyak:0.99990  Bf:100  EN:0.2170  Loss: 5.022 0.290 0.334
Ep:8500   Rew:-65.56  Avg Rew:-11.10  LR:0.00007222   Polyak:0.99990  Bf:100  EN:0.2167  Loss: 4.938 0.142 0.212
Ep:8510   Rew:88.47  Avg Rew:-12.13  LR:0.00007243   Polyak:0.99990  Bf:100  EN:0.2173  Loss: 5.099 0.184 0.165


In [ ]:
def test():  
    random_seed = 0
    n_episodes = 3
    max_timesteps = 2000
    render = True
    save_gif = True
    
    filename = "TD3_torch_{}_{}".format(env_name, random_seed)
    filename += ''
    directory = "./preTrained/td3_torch/{}".format(env_name)
    
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    
    policy = TD3(state_dim, action_dim, max_action)
    
    policy.load_actor(directory, filename)
    
    for ep in range(1, n_episodes+1):
        ep_reward = 0
        state = env.reset()
        for t in range(max_timesteps):
            action = policy.select_action(state)
            state, reward, done, _ = env.step(action)
            ep_reward += reward
            if render:
                env.render()
                if save_gif:
                    dirname = './gif/td3_torch/{}'.format(ep)
                    if not os.path.isdir(dirname):
                        os.mkdir(dirname)
                    img = env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('./gif/td3_torch/{}/{}.jpg'.format(ep,t))
            if done:
                break
            
        print('Episode: {}\tReward: {}'.format(ep, int(ep_reward)))
        ep_reward = 0
        env.close()        
                
test()
    
    